In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('../../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos > 30]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.80,4.8,1,0.00,0.8,5,0.60,3.2,0,31
7,36889,ESPORTSTMNT02_2557053,19,1,2,5,1,20,21,4,...,0.00,0.0,1,0.00,0.4,0,0.00,0.0,1,32
33,36926,ESPORTSTMNT05_2530903,19,21,20,22,23,20,21,25,...,0.00,0.0,4,0.25,2.8,0,0.00,0.0,0,34
35,36928,ESPORTSTMNT05_2530937,0,20,21,1,22,3,24,2,...,0.33,6.8,2,0.00,1.8,1,0.00,8.0,1,35
37,36930,ESPORTSTMNT05_2530947,20,2,0,20,2,1,4,3,...,0.00,3.0,2,1.00,3.0,0,0.00,0.0,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3951,42146,ESPORTSTMNT02_2884903,1,19,21,1,24,2,1,23,...,0.50,3.3,22,0.41,3.8,0,0.00,0.0,1,33
3966,41155,ESPORTSTMNT03_2788956,20,2,20,0,1,4,5,21,...,0.92,16.9,20,0.55,3.7,35,0.40,2.6,1,36
3970,41159,ESPORTSTMNT03_2789756,20,21,20,24,21,1,5,23,...,0.25,1.8,42,0.40,3.3,2,0.00,2.6,1,37
3998,40957,ESPORTSTMNT03_2785705,1,19,21,20,2,23,5,1,...,0.00,1.4,1,0.00,1.5,9,0.44,3.2,1,38


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,qtd_eventos
2,19,2,20,24,2,4,20,3,6,1,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,31
7,19,1,2,5,1,20,21,4,1,3,...,0,0.00,0.0,1,0.00,0.4,0,0.00,0.0,32
33,19,21,20,22,23,20,21,25,5,24,...,0,0.00,0.0,4,0.25,2.8,0,0.00,0.0,34
35,0,20,21,1,22,3,24,2,4,7,...,3,0.33,6.8,2,0.00,1.8,1,0.00,8.0,35
37,20,2,0,20,2,1,4,3,22,5,...,1,0.00,3.0,2,1.00,3.0,0,0.00,0.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3951,1,19,21,1,24,2,1,23,22,1,...,2,0.50,3.3,22,0.41,3.8,0,0.00,0.0,33
3966,20,2,20,0,1,4,5,21,22,7,...,12,0.92,16.9,20,0.55,3.7,35,0.40,2.6,36
3970,20,21,20,24,21,1,5,23,22,20,...,8,0.25,1.8,42,0.40,3.3,2,0.00,2.6,37
3998,1,19,21,20,2,23,5,1,24,3,...,2,0.00,1.4,1,0.00,1.5,9,0.44,3.2,38


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [11]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
# defining parameter range
param_grid={'C': [1],
            'gamma': [0.001],
            'kernel': ['rbf']}
  
grid = GridSearchCV(SVC(random_state=42),param_grid,refit=True,cv=10,n_jobs=-1)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.001, 0.1, 1], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [12]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.001, random_state=42)


In [9]:
SVCOpt = SVC(random_state=42,C=1,gamma=0.001,kernel='rbf',max_iter=100000)
SVCOpt.fit(X_train, y_train)

scores_list = []
scores_list.append({
    'Balanced Accuracy': balanced_accuracy_score(y_test,SVCOpt.predict(X_test)),
    'Precision':  precision_score(y_test,SVCOpt.predict(X_test)),
    'Recall': recall_score(y_test,SVCOpt.predict(X_test)),
    'F1-Score': f1_score(y_test,SVCOpt.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,SVCOpt.predict(X_test))
})
scores = pd.DataFrame(scores_list)

In [10]:
scores

,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,0.819892,0.769231,0.833333,0.8,0.819892
